# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [58]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report, fbeta_score, accuracy_score
import string
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dhoomilsheta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dhoomilsheta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dhoomilsheta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con=engine)
X = df.message.values
Y = df.drop(['id','message','original','genre'],axis=1)
column_names = Y.columns
Y = Y.values

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    translator = str.maketrans('', '', string.punctuation)
    # normalize
    text = text.strip().lower().translate(translator)
    tokens = word_tokenize(text)
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [72]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=100))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [74]:
y_pred = pipeline.predict(X_test)

In [75]:
print(classification_report(y_test, y_pred, target_names=column_names))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      5023
               request       0.83      0.49      0.62      1103
                 offer       0.00      0.00      0.00        23
           aid_related       0.79      0.54      0.64      2736
          medical_help       0.53      0.02      0.03       529
      medical_products       0.78      0.05      0.09       286
     search_and_rescue       0.62      0.03      0.06       169
              security       0.00      0.00      0.00       121
              military       0.71      0.05      0.09       206
                 water       0.88      0.26      0.40       433
                  food       0.83      0.40      0.54       722
               shelter       0.84      0.25      0.38       575
              clothing       0.83      0.11      0.19        92
                 money       0.80      0.03      0.05       153
        missing_people       0.50      

/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [76]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a2909da60>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
               oob_score=False, random_state=None, verbose=0,
               war

In [78]:
parameters = parameters = {
    'clf__n_estimators': [50, 100],
    'clf__max_depth': [3, 5, 7]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__n_estimators': [50, 100], 'clf__max_depth': [3, 5, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [79]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=column_names))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5023
               request       0.00      0.00      0.00      1103
                 offer       0.00      0.00      0.00        23
           aid_related       1.00      0.00      0.00      2736
          medical_help       0.00      0.00      0.00       529
      medical_products       0.00      0.00      0.00       286
     search_and_rescue       0.00      0.00      0.00       169
              security       0.00      0.00      0.00       121
              military       0.00      0.00      0.00       206
                 water       0.00      0.00      0.00       433
                  food       0.00      0.00      0.00       722
               shelter       0.00      0.00      0.00       575
              clothing       0.00      0.00      0.00        92
                 money       0.00      0.00      0.00       153
        missing_people       0.00      

/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [81]:
parameters = parameters = {
    'tfidf__use_idf': [True, False],
    'clf__n_estimators': [50, 100, 200]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'tfidf__use_idf': [True, False], 'clf__n_estimators': [50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [83]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=column_names))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.89      5023
               request       0.82      0.49      0.62      1103
                 offer       0.00      0.00      0.00        23
           aid_related       0.80      0.55      0.65      2736
          medical_help       0.56      0.02      0.03       529
      medical_products       0.76      0.05      0.09       286
     search_and_rescue       0.60      0.02      0.03       169
              security       0.00      0.00      0.00       121
              military       0.46      0.03      0.05       206
                 water       0.87      0.27      0.41       433
                  food       0.82      0.44      0.58       722
               shelter       0.84      0.25      0.38       575
              clothing       0.75      0.07      0.12        92
                 money       0.83      0.03      0.06       153
        missing_people       0.50      

### 9. Export your model as a pickle file

In [84]:
with open("../models/disaster_response_model.pkl", 'wb') as f:
    pickle.dump(pipeline, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.